In [1]:
from snowflake.snowpark import Session

from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

In [2]:
from snowflake.snowpark.functions import last_value, col
from DP_transacciones_total import get_transacciones_dp_total
from snowflake.snowpark import Window
import pandas as pd

sabana_datos_olo = session.table('SEGMENT_EVENTS.SABANA_DATOS.SABANA_DATOS_OLO').select(['EMAIL', 'PHONE']).to_pandas()

window = Window.partition_by('EMAIL').order_by('FECHA')

alt_telefonos = get_transacciones_dp_total(session).filter(col('PHONE').is_not_null()).select(['EMAIL', last_value(col('PHONE')).over(window)]).distinct().to_pandas()

In [3]:
ganadores = pd.read_excel('Lista_F1_aream.xlsx')

con_telefono = ganadores.merge(sabana_datos_olo, how = 'inner', on = 'EMAIL').to_excel('Lista_F1_aream_pt1.xlsx', index = False)

sin_telefono = ganadores[~ganadores['EMAIL'].isin(sabana_datos_olo['EMAIL'])]

con_telefono_pt2 = sin_telefono.merge(alt_telefonos, how = 'inner', on = 'EMAIL').to_excel('Lista_F1_aream_pt2.xlsx', index = False)